<a href="https://colab.research.google.com/github/igrynok/HFLU/blob/main/Assignment2_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -q

## Load libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

## Load images

In [ ]:
from google.colab import files

print("Please click 'Choose Files' and select 'dog_on_bench.jpg'")
# This will open a widget to upload files from your computer
uploaded = files.upload()

Please click 'Choose Files' and select 'dog_on_bench.jpg'


Saving dog_on_bench.jpg to dog_on_bench.jpg


# Test 1

In [ ]:
# Load the image locally
filename = "dog_on_bench.jpg"
img = cv2.imread(filename)

# Safety check
if img is None:
    print("Error: Image not found. Please upload 'dog_on_bench.jpg'")
else:
    # Convert BGR (OpenCV) to RGB (Matplotlib/Transformers)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

##  To complete the test you need to define Define the Region of Interest (ROI). It's already defined for you and you are free to adjust it.

In [ ]:
# Define your proposed rectangle
# Format: (x, y, width, height)
# You can tweak these numbers until the box looks perfect
rect = (270, 180, 245, 275)

# Draw the rectangle on a COPY of the image
# (We use a copy so we don't draw a permanent green line on the image used for processing)
img_check = img_rgb.copy()
x, y, w, h = rect
cv2.rectangle(img_check, (x, y), (x + w, y + h), (0, 255, 0), 3)

# 4. Display
plt.figure(figsize=(10, 6))
plt.imshow(img_check)
plt.title(f"ROI Check: {rect}")
plt.axis('off')
plt.show()

## 🐶 GrabCut Segmentation – Instructions

### ✅ 1: Create a zero-initialized mask
- The mask must have the same spatial shape as `img.shape[:2]`
- Use `dtype=np.uint8`

### ✅ 2: Define the Bounding Box (Region of Interest)
- Format is (x, y, width, height).
- Try to estimate a box that covers the dog but not the whole image.
- rect = (270, 180, 245, 275) # Example start point

### ✅ 3: Initialize internal models required by OpenCV
- They need to be np.zeros with shape (1, 65) and type np.float64
- bgdModel = ...
- fgdModel = ...

### ✅ 4: Run cv2.grabCut
- They need to be np.zeros with shape (1, 65) and type np.float64
- parameters: image, mask, rect, bgdModel, fgdModel, iterCount=5, mode=cv2.GC_INIT_WITH_RECT

### ✅ 5: Post-process the mask
- GrabCut modifies the mask in place with values 0,1,2,3
- Create a binary mask 'mask2' where (mask==2)|(mask==0) is 0, else 1.
- mask2 = ...

### ✅ 6: Multiply original image by mask2 to extract the dog
-  Hint: You might need to add a new axis to mask2 so it broadcasts: mask2[:, :, np.newaxis]
- result_gc = ...

### ✅ 7: Plot results

# Test 2

## 🐶 SegFormer Segmentation – Instructions

### ✅ 1: Initialize Model and Processor
- Use the stable model ID: "nvidia/segformer-b0-finetuned-ade-512-512"
- Initialize processor using SegformerImageProcessor.from_pretrained(model_name)
- Initialize model using SegformerForSemanticSegmentation.from_pretrained(model_name)

### ✅ 2: Preprocess the Image
- Use the processor to prepare the inputs
- Pass images=img_rgb and set return_tensors="pt"
- inputs = ...

### ✅ 3: Run Inference (Forward Pass)
- Use a context manager with torch.no_grad(): to disable gradient calculation (saves memory)
- Pass the inputs to the model: outputs = model(**inputs)
- Extract the raw predictions: logits = outputs.logits

### ✅ 4: Upscale Results to Original Resolution
- The model outputs small feature maps (e.g., 128x128), so we must resize them
- Use torch.nn.functional.interpolate
- Set size=img_rgb.shape[:2], mode="bilinear", and align_corners=False
upsampled_logits = ...

### ✅ 5: Generate the Class Mask
- Convert probabilities to class IDs by finding the maximum value index
- Use .argmax(dim=1) on the upsampled_logits
- Take the first item in the batch [0] to get the 2D mask
- pred_seg = ...

### ✅ 6: Visualize the Segmentation Map
- Move the tensor to CPU and convert to numpy: pred_mask = pred_seg.cpu().numpy()
- Use plt.imshow(pred_mask, cmap='tab20') (Tab20 is good for distinct categorical colors)
- Add a colorbar: plt.colorbar(label="Class ID")